# Scraping Dataset

In [9]:
!pip install requests beautifulsoup4
!pip install lxml
!pip install requests beautifulsoup4 pandas

You should consider upgrading via the 'C:\Users\HP 840 G8\Desktop\skripsi\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\HP 840 G8\Desktop\skripsi\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\HP 840 G8\Desktop\skripsi\venv\Scripts\python.exe -m pip install --upgrade pip' command.


## requests adalah library Python yang digunakan untuk mengirim permintaan HTTP dan menerima respons dari server. Fungsinya termasuk:

Mengambil data dari URL: Anda dapat menggunakan requests untuk mengunduh konten halaman web (HTML) atau API endpoint.

Fitur penting:

Mendukung berbagai metode HTTP: GET, POST, PUT, DELETE, dll.
Mendukung pengaturan headers, cookies, dan parameters.
Menangani koneksi HTTPS secara otomatis.

# ArXiv

In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import xml.etree.ElementTree as ET
import csv

In [57]:
def scrape_arxiv_by_date(query, start_date, end_date, total_results=5000, batch_size=100):
    base_url = "https://export.arxiv.org/api/query"
    all_articles = []
    
    # Membagi permintaan menjadi batch berdasarkan total hasil dan batch size
    for start in range(0, total_results, batch_size):
        params = {
            "search_query": f"all:{query}",
            "start": start,
            "max_results": batch_size,
            "sortBy": "relevance",
            "sortOrder": "descending",
            "from_date": start_date,
            "to_date": end_date
        }
        
        # Mengirim permintaan GET
        response = requests.get(base_url, params=params)
        
        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            return []

        # Menggunakan xml.etree.ElementTree untuk memparsing XML
        root = ET.fromstring(response.text)
        entries = root.findall("{http://www.w3.org/2005/Atom}entry")
        
        # Mengolah artikel-artikel yang didapatkan
        for entry in entries:
            title = entry.find("{http://www.w3.org/2005/Atom}title").text
            summary = entry.find("{http://www.w3.org/2005/Atom}summary").text
            published = entry.find("{http://www.w3.org/2005/Atom}published").text
            link = entry.find("{http://www.w3.org/2005/Atom}id").text

            # Menambahkan informasi author dan affiliation jika ada
            authors = entry.findall("{http://www.w3.org/2005/Atom}author")
            author_list = []
            institution_list = []
            for author in authors:
                name = author.find("{http://www.w3.org/2005/Atom}name")
                if name is not None:
                    author_list.append(name.text.strip())
                # Jika ada informasi institusi, dapat diambil dari elemen affiliation
                affiliation = author.find("{http://arxiv.org/schemas/atom}affiliation")
                if affiliation is not None:
                    institution_list.append(affiliation.text.strip())
            
            # Jika tidak ada affiliation yang ditemukan, bisa menambahkan "Unknown"
            if not institution_list:
                institution_list.append("Unknown")

            # Menambahkan artikel ke dalam daftar hasil
            article = {
                "Title": title,
                "Summary": summary,
                "Published": published,
                "Link": link,
                "Authors": ", ".join(author_list),
                "Institutions": ", ".join(institution_list)
            }
            all_articles.append(article)

        if len(entries) < batch_size:
            break  # Jika jumlah artikel kurang dari batch_size, hentikan permintaan lebih lanjut

    return all_articles

In [58]:
query = "neural network OR machine learning OR AI OR artificial intelligence OR data mining"
start_date = "2000-01-01"
end_date = "2024-12-29"
total_results = 5000
articles = scrape_arxiv_by_date(query, start_date, end_date, total_results=total_results)

In [59]:
# Contoh artikel
for article in articles[:5]:
    print(f"Title: {article['Title']}")
    print(f"Summary: {article['Summary']}")
    print(f"Published: {article['Published']}")
    print(f"Link: {article['Link']}")
    print(f"Authors: {article['Authors']}")
    print(f"Institutions: {article['Institutions']}\n")

Title: Behave-XAI: Deep Explainable Learning of Behavioral Representational
  Data
Summary:   According to the latest trend of artificial intelligence, AI-systems needs to
clarify regarding general,specific decisions,services provided by it. Only
consumer is satisfied, with explanation , for example, why any classification
result is the outcome of any given time. This actually motivates us using
explainable or human understandable AI for a behavioral mining scenario, where
users engagement on digital platform is determined from context, such as
emotion, activity, weather, etc. However, the output of AI-system is not always
systematically correct, and often systematically correct, but apparently
not-perfect and thereby creating confusions, such as, why the decision is
given? What is the reason underneath? In this context, we first formulate the
behavioral mining problem in deep convolutional neural network architecture.
Eventually, we apply a recursive neural network due to the presence

In [60]:
def save_to_csv(articles, filename='arxiv_articles.csv'):
    # Menentukan nama kolom yang akan disimpan ke dalam CSV
    fieldnames = ["Title", "Summary", "Published", "Link", "Authors", "Institutions"]
    
    # Membuka file CSV untuk menulis
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        # Menulis header
        writer.writeheader()
        
        # Menulis data artikel ke dalam CSV
        for article in articles:
            writer.writerow(article)
    
    print(f"Data berhasil disimpan ke {filename}")

# Menyimpan artikel ke CSV
save_to_csv(articles, 'arxiv_articles.csv')


Data berhasil disimpan ke arxiv_articles.csv


## Query 2 : bert, biLSTM, information extraction

In [26]:
import requests
import time
import pandas as pd

In [27]:
def scrape_arxiv(query, start_date, end_date, max_results=5000, batch_size=100):
    base_url = "http://export.arxiv.org/api/query"
    all_articles = []
    total_fetched = 0
    start_index = 0

    while total_fetched < max_results:
        # Build query parameters
        params = {
            "search_query": query,
            "start": start_index,
            "max_results": batch_size,
            "sortBy": "submittedDate",
            "sortOrder": "descending"
        }
        print(f"Fetching articles {start_index + 1} to {start_index + batch_size}...")
        
        # Make the request
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Error: HTTP {response.status_code}")
            break
        
        # Parse the XML response
        data = response.text
        entries = data.split("<entry>")
        if len(entries) <= 1:
            print("No more articles found.")
            break
        
        for entry in entries[1:]:
            try:
                title = entry.split("<title>")[1].split("</title>")[0].strip()
                summary = entry.split("<summary>")[1].split("</summary>")[0].strip()
                published = entry.split("<published>")[1].split("</published>")[0].strip()
                link = entry.split('<id>')[1].split('</id>')[0].strip()
                authors = ", ".join(
                    author.split("</name>")[0].split("<name>")[1].strip()
                    for author in entry.split("<author>")[1:]
                )
                all_articles.append({
                    "Title": title,
                    "Summary": summary,
                    "Published": published,
                    "Link": link,
                    "Authors": authors
                })
            except Exception as e:
                print(f"Error parsing entry: {e}")
        
        # Update counters
        num_articles = len(entries) - 1
        total_fetched += num_articles
        start_index += num_articles
        
        # Stop if no more articles
        if num_articles < batch_size:
            break
        
        # Delay to avoid overwhelming the server
        time.sleep(1)
    
    print(f"Fetched a total of {total_fetched} articles.")
    return pd.DataFrame(all_articles)


In [33]:
# Query and parameters
query = "information extraction OR BERT OR BiLSTM"
start_date = "2000-01-01"
end_date = "2024-12-31"
max_results = 5000

In [34]:
# Scrape data
articles_df = scrape_arxiv(query, start_date, end_date, max_results)

Fetching articles 1 to 100...
Fetching articles 101 to 200...
No more articles found.
Fetched a total of 100 articles.


In [30]:
# Save to CSV
articles_df.to_csv("arxiv_articles(2).csv", index=False)
print("Data saved to 'arxiv_articles(2).csv'")

Data saved to 'arxiv_articles(2).csv'


## Query 3 : NLP, Transformers, RNN, CNN, big data

In [35]:
def scrape_arxiv(query, start_date, end_date, max_results=5000, batch_size=100):
    base_url = "http://export.arxiv.org/api/query"
    all_articles = []
    total_fetched = 0
    start_index = 0

    while total_fetched < max_results:
        # Build query parameters
        params = {
            "search_query": query,
            "start": start_index,
            "max_results": batch_size,
            "sortBy": "submittedDate",
            "sortOrder": "descending"
        }
        print(f"Fetching articles {start_index + 1} to {start_index + batch_size}...")
        
        # Make the request
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Error: HTTP {response.status_code}")
            break
        
        # Parse the XML response
        data = response.text
        entries = data.split("<entry>")
        if len(entries) <= 1:
            print("No more articles found.")
            break
        
        for entry in entries[1:]:
            try:
                title = entry.split("<title>")[1].split("</title>")[0].strip()
                summary = entry.split("<summary>")[1].split("</summary>")[0].strip()
                published = entry.split("<published>")[1].split("</published>")[0].strip()
                link = entry.split('<id>')[1].split('</id>')[0].strip()
                authors = ", ".join(
                    author.split("</name>")[0].split("<name>")[1].strip()
                    for author in entry.split("<author>")[1:]
                )
                all_articles.append({
                    "Title": title,
                    "Summary": summary,
                    "Published": published,
                    "Link": link,
                    "Authors": authors
                })
            except Exception as e:
                print(f"Error parsing entry: {e}")
        
        # Update counters
        num_articles = len(entries) - 1
        total_fetched += num_articles
        start_index += num_articles
        
        # Stop if no more articles
        if num_articles < batch_size:
            break
        
        # Delay to avoid overwhelming the server
        time.sleep(1)
    
    print(f"Fetched a total of {total_fetched} articles.")
    return pd.DataFrame(all_articles)

In [47]:
# Query and parameters
query = "NLP OR RNN OR CNN OR NN OR big data OR transformers OR data structure"
start_date = "2000-01-01"
end_date = "2024-12-31"
max_results = 5000

In [48]:
# Scrape data
articles_df = scrape_arxiv(query, start_date, end_date, max_results)

Fetching articles 1 to 100...
Fetching articles 101 to 200...
No more articles found.
Fetched a total of 100 articles.


In [44]:
# Save to CSV
articles_df.to_csv("arxiv_articles(3).csv", index=False)
print("Data saved to 'arxiv_articles(3).csv'")

Data saved to 'arxiv_articles(3).csv'


## Query 4 : text mining, pattern recognition, business intelligence, data visualization 

In [53]:
# Query and parameters
query = "business inteligence OR data visualization"
start_date = "2000-01-01"
end_date = "2024-12-31"
max_results = 5000

In [54]:
# Scrape data
articles_df = scrape_arxiv(query, start_date, end_date, max_results)

Fetching articles 1 to 100...
Fetching articles 101 to 200...
Fetching articles 201 to 300...
Fetched a total of 205 articles.


In [55]:
# Save to CSV
articles_df.to_csv("arxiv_articles(4).csv", index=False)
print("Data saved to 'arxiv_articles(4).csv'")

Data saved to 'arxiv_articles(4).csv'


## Query 5 

In [63]:
# Query and parameters
query = "forecasting OR classification OR regression OR random forest OR XGBoost"
start_date = "2000-01-01"
end_date = "2024-12-31"
max_results = 5000

In [64]:
# Scrape data
articles_df = scrape_arxiv(query, start_date, end_date, max_results)

Fetching articles 1 to 100...
Fetching articles 101 to 200...
Fetching articles 201 to 300...
No more articles found.
Fetched a total of 200 articles.


In [65]:
# Save to CSV
articles_df.to_csv("arxiv_articles(5).csv", index=False)
print("Data saved to 'arxiv_articles(5).csv'")

Data saved to 'arxiv_articles(5).csv'


## Query 6

In [70]:
# Query and parameters
query = "data warehousing OR database"
start_date = "2000-01-01"
end_date = "2024-12-31"
max_results = 5000

In [71]:
# Scrape data
articles_df = scrape_arxiv(query, start_date, end_date, max_results)

Fetching articles 1 to 100...
Fetching articles 101 to 200...
Fetching articles 201 to 300...
No more articles found.
Fetched a total of 200 articles.


In [72]:
# Save to CSV
articles_df.to_csv("arxiv_articles(6).csv", index=False)
print("Data saved to 'arxiv_articles(6).csv'")

Data saved to 'arxiv_articles(6).csv'


## Query 7

In [73]:
# Query and parameters
query = "supervised learning OR unsupervised learning OR feature extraction"
start_date = "2000-01-01"
end_date = "2024-12-31"
max_results = 5000

In [74]:
# Scrape data
articles_df = scrape_arxiv(query, start_date, end_date, max_results)

Fetching articles 1 to 100...
Fetching articles 101 to 200...
Fetching articles 201 to 300...
Fetching articles 301 to 400...
Fetching articles 401 to 500...
No more articles found.
Fetched a total of 400 articles.


In [75]:
# Save to CSV
articles_df.to_csv("arxiv_articles(7).csv", index=False)
print("Data saved to 'arxiv_articles(7).csv'")

Data saved to 'arxiv_articles(7).csv'


## Query 8

In [78]:
# Query and parameters
query = "recommendation systems OR SQL OR generative AI"
start_date = "2000-01-01"
end_date = "2024-12-31"
max_results = 5000

In [79]:
# Scrape data
articles_df = scrape_arxiv(query, start_date, end_date, max_results)

Fetching articles 1 to 100...
Fetching articles 101 to 200...
Fetching articles 201 to 300...
No more articles found.
Fetched a total of 200 articles.


In [80]:
# Save to CSV
articles_df.to_csv("arxiv_articles(8).csv", index=False)
print("Data saved to 'arxiv_articles(8).csv'")

Data saved to 'arxiv_articles(8).csv'


# Combine Datset

In [81]:
import os
import pandas as pd

In [83]:
def read_and_prepare(file):
    # Baca file CSV
    df = pd.read_csv(file)
    return df

# Path ke folder CSV
path = "C:/Users/HP 840 G8/Desktop/skripsi/notebooks/data arxiv"

In [84]:
# Daftar semua file CSV
file_list = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.csv')]

In [85]:
# Baca dan persiapkan semua file
data_list = [read_and_prepare(file) for file in file_list]

In [86]:
# Gabungkan semua data
combined_data = pd.concat(data_list, ignore_index=True)

In [87]:
# Simpan hasilnya
combined_data.to_csv('arxiv_articles_dataset.csv', index=False)
print("Dataset berhasil digabungkan dan disimpan sebagai 'arxiv_articles_dataset.csv'")

Dataset berhasil digabungkan dan disimpan sebagai 'arxiv_articles_dataset.csv'


# DOAJ

In [12]:
!pip install requests beautifulsoup4 pandas

You should consider upgrading via the 'C:\Users\HP 840 G8\Desktop\skripsi\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [90]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [97]:
# DOAJ Scraping Function
def scrape_doaj(query, max_results=5000, start_year=2000, end_year=2024):
    base_url = "https://doaj.org/api/v2/search/articles"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    articles = []
    page = 1

    while len(articles) < max_results:
        print(f"Scraping DOAJ page {page}...")
        params = {
            "q": f"{query} AND created_date:[{start_year} TO {end_year}]",
            "page": page,
            "pageSize": 100  # DOAJ supports up to 100 articles per page
        }
        response = requests.get(base_url, params=params, headers=headers)

        if response.status_code != 200:
            print(f"Failed to retrieve page {page}, status code: {response.status_code}")
            break

        data = response.json()
        for result in data.get("results", []):
            articles.append({
                "Title": result.get("bibjson", {}).get("title", ""),
                "Summary": result.get("bibjson", {}).get("abstract", ""),
                "Published": result.get("created_date", ""),
                "Link": result.get("links", [{}])[0].get("url", ""),
                "Authors": ", ".join([author.get("name", "") for author in result.get("bibjson", {}).get("author", [])]),
                "Institutions": ", ".join([author.get("affiliation", "") for author in result.get("bibjson", {}).get("author", [])]),
            })

            if len(articles) >= max_results:
                break

        page += 1
        time.sleep(1)  # Prevent server overload

    return pd.DataFrame(articles)

In [98]:
if __name__ == "__main__":
    # Define queries
    query_doaj = "data science OR information extraction OR BERT OR BiLSTM"

    # Scrape DOAJ
    print("Starting DOAJ scraping...")
    doaj_data = scrape_doaj(query_doaj, max_results=5000, start_year=2000, end_year=2024)
    doaj_data.to_csv("doaj_articles.csv", index=False)
    print(f"DOAJ scraping completed. Saved {len(doaj_data)} articles to 'doaj_articles.csv'.")

Starting DOAJ scraping...
Scraping DOAJ page 1...
Failed to retrieve page 1, status code: 404
DOAJ scraping completed. Saved 0 articles to 'doaj_articles.csv'.


# ZENODO

In [15]:
import requests
import pandas as pd
import time

In [16]:
# Fungsi untuk scraping Zenodo
def scrape_zenodo(query, max_results, start_year, end_year, retries=3, delay=5):
    base_url = "https://zenodo.org/api/records/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    results = []
    total_fetched = 0
    page = 1
    
    while total_fetched < max_results:
        params = {
            "q": f"{query} AND publication_date:[{start_year} TO {end_year}]",
            "page": page,
            "size": 100  # Zenodo hanya mendukung hingga 100 artikel per halaman
        }

        attempt = 0
        while attempt < retries:
            try:
                print(f"Scraping Zenodo page {page}...")
                response = requests.get(base_url, params=params, headers=headers, timeout=30)
                
                if response.status_code == 200:
                    data = response.json()
                    for record in data['hits']['hits']:
                        title = record['metadata'].get('title', 'No Title')
                        link = record['links'].get('self', 'No Link')
                        published = record['metadata'].get('publication_date', 'No Date')
                        summary = record['metadata'].get('description', 'No Summary')
                        authors = ', '.join([author['name'] for author in record['metadata'].get('creators', [])])

                        results.append({
                            'title': title,
                            'link': link,
                            'published': published,
                            'summary': summary,
                            'authors': authors
                        })
                    
                    total_fetched += len(data['hits']['hits'])
                    if total_fetched >= max_results:
                        break
                    page += 1
                    break  # Break after successful request
                
                else:
                    print(f"Failed to retrieve page {page}, status code: {response.status_code}")
                    break  # Break on failure (e.g., status code != 200)

            except requests.exceptions.RequestException as e:
                print(f"Error occurred: {e}")
                attempt += 1
                print(f"Retrying... Attempt {attempt}/{retries}")
                time.sleep(delay)
                
    return pd.DataFrame(results)

In [17]:
# Mengambil artikel dari Zenodo dengan rentang tahun 2014-2024 dan maksimal 20000 artikel
query_zenodo = "data science OR deep learning OR natural language processing OR machine learning" 
zenodo_data = scrape_zenodo(query_zenodo, max_results=5000, start_year=2014, end_year=2024)

# Menyimpan hasil ke CSV
zenodo_data.to_csv("zenodo_articles(1).csv", index=False)
print(f"Zenodo scraping completed. Saved {len(zenodo_data)} articles to 'zenodo_articles(20.000).csv'.")

Scraping Zenodo page 1...
Scraping Zenodo page 2...
Scraping Zenodo page 3...
Scraping Zenodo page 4...
Scraping Zenodo page 5...
Scraping Zenodo page 6...
Scraping Zenodo page 7...
Scraping Zenodo page 8...
Scraping Zenodo page 9...
Scraping Zenodo page 10...
Scraping Zenodo page 11...
Scraping Zenodo page 12...
Scraping Zenodo page 13...
Scraping Zenodo page 14...
Scraping Zenodo page 15...
Scraping Zenodo page 16...
Scraping Zenodo page 17...
Scraping Zenodo page 18...
Scraping Zenodo page 19...
Scraping Zenodo page 20...
Scraping Zenodo page 21...
Scraping Zenodo page 22...
Scraping Zenodo page 23...
Scraping Zenodo page 24...
Scraping Zenodo page 25...
Scraping Zenodo page 26...
Scraping Zenodo page 27...
Scraping Zenodo page 28...
Scraping Zenodo page 29...
Scraping Zenodo page 30...
Scraping Zenodo page 31...
Scraping Zenodo page 32...
Scraping Zenodo page 33...
Scraping Zenodo page 34...
Scraping Zenodo page 35...
Scraping Zenodo page 36...
Scraping Zenodo page 37...
Scraping Z

In [19]:
# Mengambil artikel dari Zenodo dengan rentang tahun 2014-2024 
query_zenodo = "data mining OR BERT OR data structure" 
zenodo_data = scrape_zenodo(query_zenodo, max_results=5000, start_year=2014, end_year=2024)

# Menyimpan hasil ke CSV
zenodo_data.to_csv("zenodo_articles(2).csv", index=False)
print(f"Zenodo scraping completed. Saved {len(zenodo_data)} articles to 'zenodo_articles(20.000).csv'.")

Scraping Zenodo page 1...
Scraping Zenodo page 2...
Scraping Zenodo page 3...
Scraping Zenodo page 4...
Scraping Zenodo page 5...
Scraping Zenodo page 6...
Scraping Zenodo page 7...
Scraping Zenodo page 8...
Scraping Zenodo page 9...
Scraping Zenodo page 10...
Scraping Zenodo page 11...
Scraping Zenodo page 12...
Scraping Zenodo page 13...
Scraping Zenodo page 14...
Scraping Zenodo page 15...
Scraping Zenodo page 16...
Scraping Zenodo page 17...
Scraping Zenodo page 18...
Scraping Zenodo page 19...
Scraping Zenodo page 20...
Scraping Zenodo page 21...
Scraping Zenodo page 22...
Scraping Zenodo page 23...
Scraping Zenodo page 24...
Scraping Zenodo page 25...
Scraping Zenodo page 26...
Scraping Zenodo page 27...
Scraping Zenodo page 28...
Scraping Zenodo page 29...
Scraping Zenodo page 30...
Scraping Zenodo page 31...
Scraping Zenodo page 32...
Scraping Zenodo page 33...
Scraping Zenodo page 34...
Scraping Zenodo page 35...
Scraping Zenodo page 36...
Scraping Zenodo page 37...
Scraping Z

In [20]:
# Mengambil artikel dari Zenodo dengan rentang tahun 2014-2024 
query_zenodo = "artificial intelligence OR information extraction OR data visualization" 
zenodo_data = scrape_zenodo(query_zenodo, max_results=5000, start_year=2014, end_year=2024)

# Menyimpan hasil ke CSV
zenodo_data.to_csv("zenodo_articles(3).csv", index=False)
print(f"Zenodo scraping completed. Saved {len(zenodo_data)} articles to 'zenodo_articles(20.000).csv'.")

Scraping Zenodo page 1...
Scraping Zenodo page 2...
Scraping Zenodo page 3...
Scraping Zenodo page 4...
Scraping Zenodo page 5...
Scraping Zenodo page 6...
Scraping Zenodo page 7...
Scraping Zenodo page 8...
Scraping Zenodo page 9...
Scraping Zenodo page 10...
Scraping Zenodo page 11...
Scraping Zenodo page 12...
Scraping Zenodo page 13...
Scraping Zenodo page 14...
Scraping Zenodo page 15...
Scraping Zenodo page 16...
Scraping Zenodo page 17...
Scraping Zenodo page 18...
Scraping Zenodo page 19...
Scraping Zenodo page 20...
Scraping Zenodo page 21...
Scraping Zenodo page 22...
Scraping Zenodo page 23...
Scraping Zenodo page 24...
Scraping Zenodo page 25...
Scraping Zenodo page 26...
Scraping Zenodo page 27...
Scraping Zenodo page 28...
Scraping Zenodo page 29...
Scraping Zenodo page 30...
Scraping Zenodo page 31...
Scraping Zenodo page 32...
Scraping Zenodo page 33...
Scraping Zenodo page 34...
Scraping Zenodo page 35...
Scraping Zenodo page 36...
Scraping Zenodo page 37...
Scraping Z

In [21]:
# Mengambil artikel dari Zenodo dengan rentang tahun 2014-2024 
query_zenodo = "big data OR transformer OR reinforcement learning" 
zenodo_data = scrape_zenodo(query_zenodo, max_results=5000, start_year=2014, end_year=2024)

# Menyimpan hasil ke CSV
zenodo_data.to_csv("zenodo_articles(4).csv", index=False)
print(f"Zenodo scraping completed. Saved {len(zenodo_data)} articles to 'zenodo_articles.csv'.")

Scraping Zenodo page 1...
Scraping Zenodo page 2...
Scraping Zenodo page 3...
Scraping Zenodo page 4...
Scraping Zenodo page 5...
Scraping Zenodo page 6...
Scraping Zenodo page 7...
Scraping Zenodo page 8...
Scraping Zenodo page 9...
Scraping Zenodo page 10...
Scraping Zenodo page 11...
Scraping Zenodo page 12...
Scraping Zenodo page 13...
Scraping Zenodo page 14...
Scraping Zenodo page 15...
Scraping Zenodo page 16...
Scraping Zenodo page 17...
Scraping Zenodo page 18...
Scraping Zenodo page 19...
Scraping Zenodo page 20...
Scraping Zenodo page 21...
Scraping Zenodo page 22...
Scraping Zenodo page 23...
Scraping Zenodo page 24...
Scraping Zenodo page 25...
Scraping Zenodo page 26...
Scraping Zenodo page 27...
Scraping Zenodo page 28...
Scraping Zenodo page 29...
Scraping Zenodo page 30...
Scraping Zenodo page 31...
Scraping Zenodo page 32...
Scraping Zenodo page 33...
Scraping Zenodo page 34...
Scraping Zenodo page 35...
Scraping Zenodo page 36...
Scraping Zenodo page 37...
Scraping Z

In [22]:
# Mengambil artikel dari Zenodo dengan rentang tahun 2014-2024 
query_zenodo = "OR part-of-speech tagging OR text mining" 
zenodo_data = scrape_zenodo(query_zenodo, max_results=5000, start_year=2014, end_year=2024)

# Menyimpan hasil ke CSV
zenodo_data.to_csv("zenodo_articles(5).csv", index=False)
print(f"Zenodo scraping completed. Saved {len(zenodo_data)} articles to 'zenodo_articles.csv'.")

Scraping Zenodo page 1...
Scraping Zenodo page 2...
Scraping Zenodo page 3...
Scraping Zenodo page 4...
Scraping Zenodo page 5...
Scraping Zenodo page 6...
Scraping Zenodo page 7...
Scraping Zenodo page 8...
Scraping Zenodo page 9...
Scraping Zenodo page 10...
Scraping Zenodo page 11...
Scraping Zenodo page 12...
Scraping Zenodo page 13...
Scraping Zenodo page 14...
Scraping Zenodo page 15...
Scraping Zenodo page 16...
Scraping Zenodo page 17...
Scraping Zenodo page 18...
Scraping Zenodo page 19...
Scraping Zenodo page 20...
Scraping Zenodo page 21...
Scraping Zenodo page 22...
Scraping Zenodo page 23...
Scraping Zenodo page 24...
Scraping Zenodo page 25...
Scraping Zenodo page 26...
Scraping Zenodo page 27...
Scraping Zenodo page 28...
Scraping Zenodo page 29...
Scraping Zenodo page 30...
Scraping Zenodo page 31...
Scraping Zenodo page 32...
Scraping Zenodo page 33...
Scraping Zenodo page 34...
Scraping Zenodo page 35...
Scraping Zenodo page 36...
Scraping Zenodo page 37...
Scraping Z

In [23]:
# Mengambil artikel dari Zenodo dengan rentang tahun 2014-2024 
query_zenodo = "neural netowrk OR clustering OR forecasting OR random forest" 
zenodo_data = scrape_zenodo(query_zenodo, max_results=5000, start_year=2014, end_year=2024)

# Menyimpan hasil ke CSV
zenodo_data.to_csv("zenodo_articles(6).csv", index=False)
print(f"Zenodo scraping completed. Saved {len(zenodo_data)} articles to 'zenodo_articles.csv'.")

Scraping Zenodo page 1...
Scraping Zenodo page 2...
Scraping Zenodo page 3...
Scraping Zenodo page 4...
Scraping Zenodo page 5...
Scraping Zenodo page 6...
Scraping Zenodo page 7...
Scraping Zenodo page 8...
Scraping Zenodo page 9...
Scraping Zenodo page 10...
Scraping Zenodo page 11...
Scraping Zenodo page 12...
Scraping Zenodo page 13...
Scraping Zenodo page 14...
Scraping Zenodo page 15...
Scraping Zenodo page 16...
Scraping Zenodo page 17...
Scraping Zenodo page 18...
Scraping Zenodo page 19...
Scraping Zenodo page 20...
Scraping Zenodo page 21...
Scraping Zenodo page 22...
Error occurred: HTTPSConnectionPool(host='zenodo.org', port=443): Read timed out. (read timeout=30)
Retrying... Attempt 1/3
Scraping Zenodo page 22...
Scraping Zenodo page 23...
Scraping Zenodo page 24...
Scraping Zenodo page 25...
Scraping Zenodo page 26...
Scraping Zenodo page 27...
Scraping Zenodo page 28...
Scraping Zenodo page 29...
Scraping Zenodo page 30...
Scraping Zenodo page 31...
Scraping Zenodo page 

In [24]:
# Mengambil artikel dari Zenodo dengan rentang tahun 2014-2024 
query_zenodo = "decision tree OR cloud computing OR classification" 
zenodo_data = scrape_zenodo(query_zenodo, max_results=5000, start_year=2014, end_year=2024)

# Menyimpan hasil ke CSV
zenodo_data.to_csv("zenodo_articles(7).csv", index=False)
print(f"Zenodo scraping completed. Saved {len(zenodo_data)} articles to 'zenodo_articles.csv'.")

Scraping Zenodo page 1...
Scraping Zenodo page 2...
Scraping Zenodo page 3...
Scraping Zenodo page 4...
Scraping Zenodo page 5...
Scraping Zenodo page 6...
Scraping Zenodo page 7...
Scraping Zenodo page 8...
Scraping Zenodo page 9...
Scraping Zenodo page 10...
Scraping Zenodo page 11...
Scraping Zenodo page 12...
Scraping Zenodo page 13...
Scraping Zenodo page 14...
Scraping Zenodo page 15...
Scraping Zenodo page 16...
Scraping Zenodo page 17...
Scraping Zenodo page 18...
Scraping Zenodo page 19...
Scraping Zenodo page 20...
Scraping Zenodo page 21...
Scraping Zenodo page 22...
Scraping Zenodo page 23...
Scraping Zenodo page 24...
Scraping Zenodo page 25...
Scraping Zenodo page 26...
Scraping Zenodo page 27...
Scraping Zenodo page 28...
Scraping Zenodo page 29...
Scraping Zenodo page 30...
Scraping Zenodo page 31...
Scraping Zenodo page 32...
Scraping Zenodo page 33...
Scraping Zenodo page 34...
Scraping Zenodo page 35...
Scraping Zenodo page 36...
Scraping Zenodo page 37...
Scraping Z

In [25]:
# Mengambil artikel dari Zenodo dengan rentang tahun 2014-2024 
query_zenodo = "predictive modeling OR data analysis OR time series " 
zenodo_data = scrape_zenodo(query_zenodo, max_results=5000, start_year=2014, end_year=2024)

# Menyimpan hasil ke CSV
zenodo_data.to_csv("zenodo_articles(8).csv", index=False)
print(f"Zenodo scraping completed. Saved {len(zenodo_data)} articles to 'zenodo_articles.csv'.")

Scraping Zenodo page 1...
Scraping Zenodo page 2...
Scraping Zenodo page 3...
Scraping Zenodo page 4...
Scraping Zenodo page 5...
Scraping Zenodo page 6...
Scraping Zenodo page 7...
Scraping Zenodo page 8...
Scraping Zenodo page 9...
Scraping Zenodo page 10...
Scraping Zenodo page 11...
Scraping Zenodo page 12...
Scraping Zenodo page 13...
Scraping Zenodo page 14...
Scraping Zenodo page 15...
Scraping Zenodo page 16...
Scraping Zenodo page 17...
Scraping Zenodo page 18...
Scraping Zenodo page 19...
Scraping Zenodo page 20...
Scraping Zenodo page 21...
Scraping Zenodo page 22...
Scraping Zenodo page 23...
Scraping Zenodo page 24...
Scraping Zenodo page 25...
Scraping Zenodo page 26...
Scraping Zenodo page 27...
Scraping Zenodo page 28...
Scraping Zenodo page 29...
Scraping Zenodo page 30...
Scraping Zenodo page 31...
Scraping Zenodo page 32...
Scraping Zenodo page 33...
Scraping Zenodo page 34...
Scraping Zenodo page 35...
Scraping Zenodo page 36...
Scraping Zenodo page 37...
Scraping Z

# Combine Zenodo Dataset

In [26]:
import pandas as pd
import os

In [27]:
# Folder yang berisi file CSV
folder_path = "C:/Users/HP 840 G8/Desktop/skripsi/notebooks/data zenodo"

In [28]:
# Membuat daftar file CSV
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

In [29]:
# Membaca dan menggabungkan semua file CSV
combined_data = pd.DataFrame()  # DataFrame kosong
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)  # Membaca setiap file CSV
    combined_data = pd.concat([combined_data, data], ignore_index=True) 

In [30]:
# Menyimpan hasil gabungan ke file CSV baru
output_file = "articles_zenodo_dataset.csv"
combined_data.to_csv(output_file, index=False)

print(f"File berhasil digabungkan dan disimpan sebagai '{output_file}'.")

File berhasil digabungkan dan disimpan sebagai 'articles_zenodo_dataset.csv'.


## Cleaning Data Zenodo

In [31]:
import pandas as pd

# Membaca file CSV
data = pd.read_csv('articles_zenodo_dataset.csv')

# Menampilkan 5 baris pertama
data.head()

,title,link,published,summary,authors
0,Hate Speech Detection in Twitter: Natural Lang...,https://zenodo.org/api/records/11178221,2023-11-22,<p>The proliferation of social media platforms...,"Egode, Kelly, Oraegbunam, Linda, Oyatunji, Ade..."
1,Job Prediction based on Skills and Years of ex...,https://zenodo.org/api/records/10325357,2023-12-09,<h2><strong>Purpose:</strong></h2><p>This data...,"Pettugani, Hotragn"
2,Improving Medical Coding Processes with Data A...,https://zenodo.org/api/records/11079994,2021-12-31,"<p><span>In this study, we explore the pressin...",Arun Chandramouli
3,Deep Learning: Theoretical and Practical Approach,https://zenodo.org/api/records/6672318,2022-06-24,<p><strong>Deep Learning Course Book In Persia...,Sina Ranjbar Kooh Farhadi
4,Application of NLP and Machine Learning for Me...,https://zenodo.org/api/records/8036627,2022-08-30,<p><strong>Abstract: </strong>Humans&#39; most...,"Trinayan Borah, S. Ganesh Kumar"


In [32]:
# Memeriksa informasi data

## Menampilkan informasi data
print(data.info())

## Menampilkan statistik deskriptif
print(data.describe())

## Melihat apakah ada nilai yang hilang (NaN)
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      40000 non-null  object
 1   link       40000 non-null  object
 2   published  40000 non-null  object
 3   summary    39924 non-null  object
 4   authors    39999 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB
None
                                                    title  \
count                                               40000   
unique                                              35167   
top     Reproduction of: Retiring Adult - New Datasets...   
freq                                                    8   

                                          link   published  \
count                                    40000       40000   
unique                                   35942        3852   
top     https://zenodo.org/api/records/8050516  2018-04-01   
freq                

In [34]:
# Menampilkan jumlah baris sebelum penghapusan
print("Jumlah baris sebelum penghapusan:", len(data))

Jumlah baris sebelum penghapusan: 40000


In [35]:
# Menghapus baris yang memiliki nilai kosong di kolom 'summary' atau 'authors'
data_cleaned = data.dropna(subset=['summary', 'authors'])

In [36]:
# Menampilkan jumlah baris setelah penghapusan
print("Jumlah baris setelah penghapusan:", len(data_cleaned))

Jumlah baris setelah penghapusan: 39923


In [37]:
#  Menyimpan data yang telah dibersihkan ke file baru
data_cleaned.to_csv('zenodo_articles_cleaned.csv', index=False)

print("Data yang telah dibersihkan disimpan ke 'zenodo_articles_cleaned.csv'.")

Data yang telah dibersihkan disimpan ke 'zenodo_articles_cleaned.csv'.


## Penyamaan format published

In [38]:
# Menampilkan beberapa baris sebelum pengubahan format tanggal
print("Sebelum format disamakan:")
print(data['published'].head())

Sebelum format disamakan:
0    2023-11-22
1    2023-12-09
2    2021-12-31
3    2022-06-24
4    2022-08-30
Name: published, dtype: object


In [39]:
# Menambahkan tanggal default untuk format yang tidak lengkap (hanya tahun dan bulan)
data['published'] = data['published'].apply(
    lambda x: f"{x}-01" if isinstance(x, str) and len(x) == 7 else x
)

In [40]:
# Mengonversi ke format datetime
data['published'] = pd.to_datetime(data['published'], errors='coerce')

In [42]:
# Menampilkan baris dengan nilai NaT setelah konversi
missing_dates = data[data['published'].isna()]
if not missing_dates.empty:
    print("\nBaris dengan tanggal tidak valid:")
    print(missing_dates)


Baris dengan tanggal tidak valid:
                                                   title  \
136    Near-real-time Country-wide Estimation of Susc...   
148    Zero-Shot Cross-lingual Aphasia Detection usin...   
354    Conquering catastrophic forgetting in machine ...   
608    Advanced Data Analysis for Machine Learning-po...   
634    Enabling Deep Learning on IoT Devices: A Compr...   
...                                                  ...   
39620  Data pertaining to 'Dunedin groundwater monito...   
39659  A novel feature engineering approach for high-...   
39747  DUAREM: Full-scale experimental validation of ...   
39913  A transfer entropy analysis of leader-follower...   
39984  Social skills, autism and technologies: An ana...   

                                          link published  \
136     https://zenodo.org/api/records/8099812       NaT   
148    https://zenodo.org/api/records/10089640       NaT   
354    https://zenodo.org/api/records/14057039       NaT   
608 

In [43]:
# Menyimpan data yang telah diperbarui ke file baru
data.to_csv('zenodo_articles_uniform_dates.csv', index=False)

print("\nData dengan format tanggal yang disamakan telah disimpan ke 'zenodo_articles_uniform_dates.csv'.")


Data dengan format tanggal yang disamakan telah disimpan ke 'zenodo_articles_uniform_dates.csv'.
